In [1]:
!pip install torch torchvision pycocotools tqdm pillow torchmetrics
!apt install tree -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
# Célula 2 (VERIFIQUE SE ESTÁ ASSIM)

# Importar bibliotecas
import torch
import torchvision
from torchvision import tv_tensors
from torchvision.models.detection import SSD300_VGG16_Weights
from torchvision.models.detection.ssd import SSDHead, SSDClassificationHead
from torchvision.datasets import CocoDetection
from torch.utils.data import Dataset, DataLoader
import os
from torchmetrics.detection import MeanAveragePrecision
import torchvision.transforms.v2 as T
from PIL import Image
from tqdm import tqdm

In [3]:
LOCAL_DATASET_PATH = "/kaggle/input/robocup-coco"

# (Opcional) Mostra a estrutura descompactada
!tree -L 3 "{LOCAL_DATASET_PATH}"

/kaggle/input/robocup-coco
└── robocup_coco
    ├── README.dataset.txt
    ├── README.roboflow.txt
    ├── test
    │   ├── _annotations.coco.json
    │   └── Imagens
    ├── train
    │   ├── _annotations.coco.json
    │   └── Imagens
    └── valid
        ├── _annotations.coco.json
        └── Imagens

7 directories, 5 files


In [4]:
# Configurações
DATASET_PATH = LOCAL_DATASET_PATH + "/robocup_coco"
NUM_EPOCHS = 30
BATCH_SIZE = 4
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Usando o dispositivo: {DEVICE}")

# Configurações checkpointing
# Para Colab (aponte para seu Google Drive):
# CHECKPOINT_DIR = "/content/drive/MyDrive/UFU/MSc. 2025-2/Top IA - Análise de Imagem e Vídeo/Colab/Trabalho Final/ssd_vgg16_augmented_checkpoint"
# Para Kaggle (use o diretório de trabalho):
CHECKPOINT_DIR = "/kaggle/working/ssd_vgg16_augmented_checkpoint"

os.makedirs(CHECKPOINT_DIR, exist_ok=True)
CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, "checkpoint.pth")
BEST_MODEL_PATH = os.path.join(CHECKPOINT_DIR, "best_model.pth")
print(f"Checkpoints serão salvos em: {CHECKPOINT_DIR}")

Usando o dispositivo: cuda
Checkpoints serão salvos em: /kaggle/working/ssd_vgg16_augmented_checkpoint


In [5]:
# Criar modelo
def get_model(num_classes):
    """Cria um modelo SSD300 com backbone VGG16."""
    
    # 1. Carrega o modelo SSD300 com backbone VGG16, pré-treinado no COCO
    weights = SSD300_VGG16_Weights.DEFAULT
    model = torchvision.models.detection.ssd300_vgg16(weights=weights)

    # 2. Obter os parâmetros da cabeça de classificação original
    # (Estes são os canais de saída do VGG16 nos quais o SSD faz predições)
    in_channels = [512, 1024, 512, 256, 256, 256] 
    num_anchors = model.anchor_generator.num_anchors_per_location()
    
    # 3. Criar uma nova cabeça de classificação com o número de classes desejado
    # (num_classes já inclui o background)
    new_head = SSDClassificationHead(in_channels=in_channels, num_anchors=num_anchors, num_classes=num_classes)

    # 4. Substituir a cabeça do modelo
    model.head.classification_head = new_head
    
    return model

In [6]:
# 1. Onde seu script de treino espera que os checkpoints estejam
# (Este caminho DEVE ser o mesmo da Célula 5)
KAGGLE_WORKING_DIR = CHECKPOINT_DIR # CHECKPOINT_DIR já foi definido na Célula 5

# 2. Onde o Kaggle montou seu Dataset de checkpoints
# (Mude 'robocup-mobilenet-checkpoint' se você deu um nome diferente)
INPUT_CHECKPOINT_DIR = "/kaggle/input/checkpoint-ssd-vgg"

# 3. Os arquivos que queremos copiar
CHECKPOINT_FILE = "checkpoint.pth"
BEST_MODEL_FILE = "best_model.pth"

# 4. Copia os arquivos
print("Copiando checkpoints pré-treinados do Kaggle Input para o Kaggle Working...")

# Copia o checkpoint principal
src_path = os.path.join(INPUT_CHECKPOINT_DIR, CHECKPOINT_FILE)
dst_path = os.path.join(KAGGLE_WORKING_DIR, CHECKPOINT_FILE)
if os.path.exists(src_path):
    !cp "{src_path}" "{dst_path}"
    print(f"Arquivo {CHECKPOINT_FILE} copiado.")
else:
    print(f"Aviso: {CHECKPOINT_FILE} não encontrado em {INPUT_CHECKPOINT_DIR}")

# Copia o 'best_model.pth' para que 'best_map' seja carregado corretamente
src_path_best = os.path.join(INPUT_CHECKPOINT_DIR, BEST_MODEL_FILE)
dst_path_best = os.path.join(KAGGLE_WORKING_DIR, BEST_MODEL_FILE)
if os.path.exists(src_path_best):
    !cp "{src_path_best}" "{dst_path_best}"
    print(f"Arquivo {BEST_MODEL_FILE} copiado.")

print("Cópia concluída. Verificando arquivos:")
!ls -l "{KAGGLE_WORKING_DIR}"

Copiando checkpoints pré-treinados do Kaggle Input para o Kaggle Working...
Arquivo checkpoint.pth copiado.
Cópia concluída. Verificando arquivos:
total 203164
-rw-r--r-- 1 root root 208038536 Oct 19 18:27 checkpoint.pth


In [7]:
# Médias e desvios padrão do ImageNet (usados pelo VGG16)

# Médias e desvios padrão do ImageNet (usados pelo VGG16)
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

def get_transform(train):
    """Define as transformações/data augmentation PARA SSD300+VGG16."""
    transforms = []
    
    # --- 1. Augmentations (operam em PIL.Image) ---
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
        transforms.append(T.RandomAffine(degrees=10, translate=(0.05, 0.05)))
        transforms.append(T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1))
        transforms.append(T.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.)))

    # --- 2. Conversão (PIL -> Tensor) ---
    transforms.append(T.ToImage()) 
    transforms.append(T.ToDtype(torch.float32, scale=True)) # Normaliza para [0.0, 1.0]

    # --- 3. Augmentations (operam em Tensor) ---
    if train:
        transforms.append(T.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3)))

    # --- 4. Transformações OBRIGATÓRIAS (REINTRODUZIDAS) ---
    # O SSD300 espera imagens 300x300
    transforms.append(T.Resize([300, 300], antialias=True)) 
    
    # Remove caixas que podem ter ficado inválidas APÓS o resize/affine
    transforms.append(T.SanitizeBoundingBoxes()) 
    
    # Normaliza com as médias do VGG16 (necessário para pesos pré-treinados)
    transforms.append(T.Normalize(mean=MEAN, std=STD))

    return T.Compose(transforms)

# Necessária porque, em detecção de objetos, os dados dentro de um lote (batch)
# não têm o mesmo tamanho (são "irregulares": imagens com tamanhos diferentes, nro. de boxes diferentes).
def collate_fn(batch):
    """
    Função 'collate' para o DataLoader, para lidar com lotes de dados.
    Lógica personalizada para criar um batch.
    """
    return tuple(zip(*batch))

In [8]:
class RobocupCocoDataset(CocoDetection):
    def __init__(self, root, annFile, transforms=None, junk_category_name=None):
        # 1. Inicializa a classe CocoDetection
        # Não passa as transformações para o 'super' ainda,
        # pois as aplicaremos manualmente no final
        super().__init__(root, annFile)

        # 2. Armazena as transformações
        self.transforms = transforms

        # 3. Aplica o patch (remove junk classes)
        if junk_category_name:
            print(f"Iniciando 'patch' do dataset para remover '{junk_category_name}'...")
            self._patch_dataset(junk_category_name)
            print("Patch concluído.")

    def _patch_dataset(self, junk_category_name):
        junk_cat_id = None
        for cat_id, cat_info in list(self.coco.cats.items()):
            if cat_info['name'] == junk_category_name:
                junk_cat_id = cat_id
                del self.coco.cats[cat_id]
                break

        if junk_cat_id is None:
            print(f"Aviso: Categoria '{junk_category_name}' não encontrada.")
            return

        print(f"Categoria 'junk' encontrada com ID: {junk_cat_id}. Removendo...")
        anns_to_remove = []
        for ann_id, ann in self.coco.anns.items():
            if ann['category_id'] == junk_cat_id:
                anns_to_remove.append(ann_id)

        for ann_id in anns_to_remove:
            del self.coco.anns[ann_id]

        for img_id, ann_ids in self.coco.imgToAnns.items():
            self.coco.imgToAnns[img_id] = [ann_id for ann_id in ann_ids if ann_id not in anns_to_remove]

        original_image_count = len(self.ids)
        self.ids = [img_id for img_id in self.ids if len(self.coco.imgToAnns[img_id]) > 0]
        new_image_count = len(self.ids)

        print(f"{len(anns_to_remove)} anotações 'junk' removidas.")
        print(f"{original_image_count - new_image_count} imagens removidas por ficarem vazias.")

    def __getitem__(self, index):
        # 1. Pega os dados brutos usando a lógica interna do CocoDetection
        img_id = self.ids[index]
        image = self._load_image(img_id)      # Carrega a Imagem PIL
        target_list = self._load_target(img_id) # Carrega a LISTA de anotações

        # 2. Converte a lista de anotações (target_list) no dicionário (target_dict)
        boxes = []
        labels = []
        areas = []
        iscrowd = []

        for ann in target_list:
            # O formato COCO 'bbox' é [x, y, width, height]
            # O formato 'torchvision' 'boxes' é [x1, y1, x2, y2]
            xmin = ann['bbox'][0]
            ymin = ann['bbox'][1]
            xmax = xmin + ann['bbox'][2]
            ymax = ymin + ann['bbox'][3]
            boxes.append([xmin, ymin, xmax, ymax])

            labels.append(ann['category_id'])
            areas.append(ann['area'])
            iscrowd.append(ann['iscrowd'])

        # 3. Converte para Tensores
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        areas = torch.as_tensor(areas, dtype=torch.float32)
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)
        image_id = torch.tensor([img_id])

        # 4. Monta o dicionário de target final
        target_dict = {}
        
        # Envolve o tensor de caixas na classe BoundingBoxes
        # Isso informa à API v2 o formato e o tamanho da imagem (canvas)
        target_dict["boxes"] = tv_tensors.BoundingBoxes(
            boxes,
            format=tv_tensors.BoundingBoxFormat.XYXY,
            canvas_size=image.size[::-1]  # (height, width)
        )
        
        target_dict["labels"] = labels
        target_dict["image_id"] = image_id
        target_dict["area"] = areas
        target_dict["iscrowd"] = iscrowd

        # 5. Aplica as transformações (agora elas recebem a img e o target_dict)
        if self.transforms is not None:
            image, target_dict = self.transforms(image, target_dict)

        return image, target_dict

# Treinamento

In [9]:
# Caminhos para os conjuntos de treino e validação
train_dir = os.path.join(DATASET_PATH, "train")
valid_dir = os.path.join(DATASET_PATH, "valid")

# Caminhos específicos para imagens e anotações de TREINO
train_img_dir = os.path.join(train_dir, "Imagens")
train_ann_file = os.path.join(train_dir, "_annotations.coco.json")

# Caminhos específicos para imagens e anotações de VALIDAÇÃO
valid_img_dir = os.path.join(valid_dir, "Imagens")
valid_ann_file = os.path.join(valid_dir, "_annotations.coco.json")

JUNK_NAME = 'Axis2-Bearing2-Housing-Motor2-FRQP'

# 1. Criar o Dataset de Treino
train_dataset = RobocupCocoDataset(
    root=train_img_dir,
    annFile=train_ann_file,
    transforms=get_transform(train=True),
    junk_category_name=JUNK_NAME
)

# 2. Criar o Dataset de Validação
valid_dataset = RobocupCocoDataset(
    root=valid_img_dir,
    annFile=valid_ann_file,
    transforms=get_transform(train=False),
    junk_category_name=JUNK_NAME
)

print(f"Dataset de treino carregado com {len(train_dataset)} imagens.")
print(f"Dataset de validação carregado com {len(valid_dataset)} imagens.")

num_classes_from_dataset = len(train_dataset.coco.cats)
NUM_CLASSES = num_classes_from_dataset + 1 # +1 para o background
print(f"Número de classes detectado: {num_classes_from_dataset} (+1 background = {NUM_CLASSES})")

# Criando o mapa de classes para usar na inferência (opcional, mas útil)
CLASS_NAMES = {cat_id: info['name'] for cat_id, info in train_dataset.coco.cats.items()}

print("Categorias: ")
print(train_dataset.coco.cats)
model_class_map = {i: cat['name'] for i, cat in train_dataset.coco.cats.items()}
print("Mapa de classes para o modelo:")
print(model_class_map)

# --- DATALOADERS ---
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, collate_fn=collate_fn
)
valid_loader = DataLoader(
    valid_dataset, batch_size=1, shuffle=False, num_workers=2, collate_fn=collate_fn
)

loading annotations into memory...
Done (t=1.27s)
creating index...
index created!
Iniciando 'patch' do dataset para remover 'Axis2-Bearing2-Housing-Motor2-FRQP'...
Categoria 'junk' encontrada com ID: 0. Removendo...
0 anotações 'junk' removidas.
1960 imagens removidas por ficarem vazias.
Patch concluído.
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
Iniciando 'patch' do dataset para remover 'Axis2-Bearing2-Housing-Motor2-FRQP'...
Categoria 'junk' encontrada com ID: 0. Removendo...
0 anotações 'junk' removidas.
98 imagens removidas por ficarem vazias.
Patch concluído.
Dataset de treino carregado com 32489 imagens.
Dataset de validação carregado com 1628 imagens.
Número de classes detectado: 18 (+1 background = 19)
Categorias: 
{1: {'id': 1, 'name': 'AllenKey', 'supercategory': 'Axis2-Bearing2-Housing-Motor2-FRQP'}, 2: {'id': 2, 'name': 'Axis2', 'supercategory': 'Axis2-Bearing2-Housing-Motor2-FRQP'}, 3: {'id': 3, 'name': 'Bearing2', 'supercategory': 

In [10]:
# Defina o número de classes (suas classes + 1 para o fundo)
model = get_model(NUM_CLASSES)

# Mover o modelo para a GPU, se disponível
model.to(DEVICE)

# Definir otimizador
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Agendador de taxa de aprendizado
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

Downloading: "https://download.pytorch.org/models/ssd300_vgg16_coco-b556d3b4.pth" to /root/.cache/torch/hub/checkpoints/ssd300_vgg16_coco-b556d3b4.pth
100%|██████████| 136M/136M [00:00<00:00, 176MB/s]


In [11]:
# --- LÓGICA PARA CARREGAR O CHECKPOINT (RESUMIR) ---
start_epoch = 0
best_map = 0.0

# Inicializa listas para armazenar o histórico
train_loss_history = []
val_map_history = []

if os.path.exists(CHECKPOINT_PATH):
    print(f"Carregando checkpoint de '{CHECKPOINT_PATH}'...")
    checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVICE)

    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1 # Começa da *próxima* época
    best_map = checkpoint['best_map']

    # Carrega o histórico salvo para continuar
    if 'train_loss_history' in checkpoint:
        train_loss_history = checkpoint['train_loss_history']
    if 'val_map_history' in checkpoint:
        val_map_history = checkpoint['val_map_history']

    print(f"Checkpoint carregado. Resumindo da época {start_epoch}")
    print(f"Melhor mAP até agora: {best_map:.4f}")
else:
    print("Nenhum checkpoint encontrado. Começando treinamento do zero.")

Carregando checkpoint de '/kaggle/working/ssd_vgg16_augmented_checkpoint/checkpoint.pth'...
Checkpoint carregado. Resumindo da época 24
Melhor mAP até agora: 0.0000


In [12]:
# Loop de treinamento
for epoch in range(start_epoch, NUM_EPOCHS):
    model.train() # Coloca o modelo em modo de treinamento
    epoch_loss = 0

    progress_bar = tqdm(train_loader, desc=f"Época {epoch+1}/{NUM_EPOCHS}", unit="batch")

    for images, targets in progress_bar:
        # Mover dados para o device correto
        images = list(image.to(DEVICE) for image in images)

        # 'targets' é um tuple de dicionários
        # Precisamos converter o 'tv_tensor' de volta para 'tensor'
        
        targets_list = []
        for t_dict in targets: # t_dict é o target de UMA imagem
            processed_dict = {}
            
            for k, v in t_dict.items():
                if k == "boxes":
                    # --- Processa "boxes" ---
                    # O modelo em modo .train() espera o tensor puro (.data)
                    if isinstance(v, tv_tensors.BoundingBoxes):
                        processed_dict[k] = v.data.to(device=DEVICE, dtype=torch.float32)
                    elif isinstance(v, torch.Tensor):
                         # Fallback: Se já for tensor
                         processed_dict[k] = v.to(device=DEVICE, dtype=torch.float32)
                
                elif isinstance(v, torch.Tensor):
                    # --- Processa TODAS as outras chaves ---
                    # Copia 'labels', 'area', 'iscrowd', 'image_id' etc. para a GPU
                    processed_dict[k] = v.to(device=DEVICE)
                
                # Outros tipos (ex: metadados que não são tensores) são ignorados
            
            # Adiciona à lista apenas se tiver as chaves essenciais
            if "boxes" in processed_dict and "labels" in processed_dict:
                targets_list.append(processed_dict)
            else:
                img_id_val = t_dict.get("image_id", "Desconhecido")
                if isinstance(img_id_val, torch.Tensor):
                    img_id_val = img_id_val.item()
                print(f"Aviso: Target ignorado por falta de 'boxes' ou 'labels' após processamento. Img ID: {img_id_val}")
                
        targets = targets_list
        # print(targets_list)
        # O modelo retorna um dicionário de perdas em modo de treino
        loss_dict = model(images, targets)

        # Somar todas as perdas
        losses = sum(loss for loss in loss_dict.values())

        # Backpropagation
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        batch_loss = losses.item()
        epoch_loss += batch_loss
        progress_bar.set_postfix(loss=batch_loss)

    # Calcula e salva a perda da época
    train_loss = epoch_loss/len(train_loader)
    train_loss_history.append(train_loss)

    # Atualiza o agendador de taxa de aprendizado
    lr_scheduler.step()

    print(f"Época {epoch+1}/{NUM_EPOCHS}, Perda: {epoch_loss/len(train_loader)}")

    # --- VALIDAÇÃO (ESSENCIAL PARA SALVAR O MELHOR MODELO) ---
    model.eval()
    metric = MeanAveragePrecision()
    with torch.no_grad():
        for images, targets in valid_loader:
            images = list(image.to(DEVICE) for image in images)
            
            # --- CORREÇÃO: Processar targets de validação manualmente ---
            targets_list_val = []
            for t_dict in targets:
                processed_dict = {}
                for k, v in t_dict.items():
                    if k == "boxes":
                        if isinstance(v, tv_tensors.BoundingBoxes):
                            # A métrica espera um tensor puro (.data)
                            processed_dict[k] = v.data.to(device=DEVICE, dtype=torch.float32)
                        elif isinstance(v, torch.Tensor):
                             processed_dict[k] = v.to(device=DEVICE, dtype=torch.float32)
                    elif isinstance(v, torch.Tensor):
                        # Copia 'labels', 'area', 'iscrowd', 'image_id' etc.
                        processed_dict[k] = v.to(device=DEVICE)
                targets_list_val.append(processed_dict)
            
            targets = targets_list_val # Usa a lista processada
            predictions = model(images)
            metric.update(predictions, targets)

    val_metrics = metric.compute()
    val_map = val_metrics['map'].item() # Pega o mAP principal (IoU de 0.5 a 0.95)
    # Salva a métrica de validação da época
    val_map_history.append(val_map)

    # Imprime o dicionário completo de métricas
    print("Métricas de Validação:")
    for k, v in val_metrics.items():
        # 'v' é sempre um tensor. Verificamos se ele tem 1 elemento (escalar)
        if v.numel() == 1:
            # Se for escalar, usamos .item() e formatamos
            print(f"  {k}: {v.item():.4f}")
        else:
            # Se tiver >1 elemento (ex: as métricas 'per_class' de 18 elementos)
            # Apenas imprimimos o tensor. Não usamos .item()
            print(f"  {k}: {v}")


    print(f"Época {epoch+1}/{NUM_EPOCHS} - "
            f"Perda Treino: {train_loss:.4f} - "
            f"mAP Validação: {val_map:.4f}")
    # --- LÓGICA PARA SALVAR O CHECKPOINT ---

    # 1. Salvar o melhor modelo (para inferência futura)
    if val_map > best_map:
        best_map = val_map
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f"Novo melhor modelo salvo com mAP: {best_map:.4f} em '{BEST_MODEL_PATH}'")

    # 2. Salvar o checkpoint da última época (para resumir)
    checkpoint_data = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': lr_scheduler.state_dict(),
        'best_map': best_map,
        'train_loss_history': train_loss_history,
        'val_map_history': val_map_history
    }
    torch.save(checkpoint_data, CHECKPOINT_PATH)
    print(f"Checkpoint da época {epoch+1} salvo em '{CHECKPOINT_PATH}'")

print("Treinamento concluído.")

# Salvar o modelo treinado
torch.save(model.state_dict(), 'ssd_vgg.pth')
print("Modelo salvo")

Época 25/30:   0%|          | 0/8123 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_augment.py:93: UserWarning: RandomErasing() is currently passing through inputs of type tv_tensors.BoundingBoxes. This will likely change in the future.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_augment.py:93: UserWarning: RandomErasing() is currently passing through inputs of type tv_tensors.BoundingBoxes. This will likely change in the future.
  warnings.warn(
Época 25/30: 100%|██████████| 8123/8123 [26:26<00:00,  5.12batch/s, loss=nan]

Época 25/30, Perda: nan


Métricas de Validação:
  map: 0.0000
  map_50: 0.0000
  map_75: 0.0000
  map_small: 0.0000
  map_medium: 0.0000
  map_large: 0.0000
  mar_1: 0.0000
  mar_10: 0.0000
  mar_100: 0.0000
  mar_small: 0.0000
  mar_medium: 0.0000
  mar_large: 0.0000
  map_per_class: -1.0000
  mar_100_per_class: -1.0000
  classes: tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
       dtype=torch.int32)
Época 25/30 - Perda Treino: nan - mAP Validação: 0.0000
Checkpoint da época 25 salvo em '/kaggle/working/ssd_vgg16_augmented_checkpoint/checkpoint.pth'


Época 26/30:   0%|          | 0/8123 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_augment.py:93: UserWarning: RandomErasing() is currently passing through inputs of type tv_tensors.BoundingBoxes. This will likely change in the future.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_augment.py:93: UserWarning: RandomErasing() is currently passing through inputs of type tv_tensors.BoundingBoxes. This will likely change in the future.
  warnings.warn(
Época 26/30: 100%|██████████| 8123/8123 [31:07<00:00,  4.35batch/s, loss=nan]

Época 26/30, Perda: nan


...

Época 29/30, Perda: nan


Métricas de Validação:
  map: 0.0000
  map_50: 0.0000
  map_75: 0.0000
  map_small: 0.0000
  map_medium: 0.0000
  map_large: 0.0000
  mar_1: 0.0000
  mar_10: 0.0000
  mar_100: 0.0000
  mar_small: 0.0000
  mar_medium: 0.0000
  mar_large: 0.0000
  map_per_class: -1.0000
  mar_100_per_class: -1.0000
  classes: tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
       dtype=torch.int32)
Época 29/30 - Perda Treino: nan - mAP Validação: 0.0000
Checkpoint da época 29 salvo em '/kaggle/working/ssd_vgg16_augmented_checkpoint/checkpoint.pth'


Época 30/30:   0%|          | 0/8123 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_augment.py:93: UserWarning: RandomErasing() is currently passing through inputs of type tv_tensors.BoundingBoxes. This will likely change in the future.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_augment.py:93: UserWarning: RandomErasing() is currently passing through inputs of type tv_tensors.BoundingBoxes. This will likely change in the future.
  warnings.warn(
Época 30/30: 100%|██████████| 8123/8123 [31:49<00:00,  4.25batch/s, loss=nan]

Época 30/30, Perda: nan


Métricas de Validação:
  map: 0.0000
  map_50: 0.0000
  map_75: 0.0000
  map_small: 0.0000
  map_medium: 0.0000
  map_large: 0.0000
  mar_1: 0.0000
  mar_10: 0.0000
  mar_100: 0.0000
  mar_small: 0.0000
  mar_medium: 0.0000
  mar_large: 0.0000
  map_per_class: -1.0000
  mar_100_per_class: -1.0000
  classes: tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
       dtype=torch.int32)
Época 30/30 - Perda Treino: nan - mAP Validação: 0.0000
Checkpoint da época 30 salvo em '/kaggle/working/ssd_vgg16_augmented_checkpoint/checkpoint.pth'
Treinamento concluído.
Modelo salvo


# Inferência (e FPS)

In [13]:
import time

# --- Carregar o MELHOR modelo salvo ---
print(f"Carregando o melhor modelo de '{BEST_MODEL_PATH}' para medição de FPS...")
# 1. Recriar a arquitetura do modelo
model = get_model(NUM_CLASSES)

# 2. Carregar os pesos salvos
model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval() # MUITO IMPORTANTE: colocar em modo de avaliação

print("Modelo carregado. Iniciando medição de FPS...")

# --- Loop de Medição ---
# Usaremos o valid_loader, mas sem calcular métricas, apenas o tempo
total_time = 0
image_count = 0
inference_loader = DataLoader(
    valid_dataset, batch_size=1, shuffle=False, num_workers=2, collate_fn=collate_fn
)

with torch.no_grad():
    for images, targets in inference_loader:
        images = list(image.to(DEVICE) for image in images)

        # Sincroniza a GPU para uma medição de tempo precisa (se estiver usando cuda)
        if DEVICE.type == 'cuda':
            torch.cuda.synchronize()

        start_time = time.time()

        # Apenas executa a inferência
        predictions = model(images)

        if DEVICE.type == 'cuda':
            torch.cuda.synchronize()

        end_time = time.time()

        total_time += (end_time - start_time)
        image_count += len(images)

# --- Calcular Resultados ---
avg_inference_time = total_time / image_count
fps = 1.0 / avg_inference_time

print("--- Resultados de Performance ---")
print(f"Total de imagens processadas: {image_count}")
print(f"Tempo total de inferência: {total_time:.2f} segundos")
print(f"Tempo médio por imagem: {avg_inference_time * 1000:.2f} ms") # Converte para milissegundos
print(f"FPS (Frames Per Second): {fps:.2f}")

Carregando o melhor modelo de '/kaggle/working/ssd_vgg16_augmented_checkpoint/best_model.pth' para medição de FPS...


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/ssd_vgg16_augmented_checkpoint/best_model.pth'

# Gráficos

In [ ]:
import matplotlib.pyplot as plt

# Carrega o checkpoint final para garantir que temos o histórico completo
# (Mesmo que o treinamento tenha acabado de rodar, é uma boa prática)
final_checkpoint = torch.load(CHECKPOINT_PATH)
loss_hist = final_checkpoint['train_loss_history']
map_hist = final_checkpoint['val_map_history']

# Cria a figura e o primeiro eixo (Perda de Treino)
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot da Perda de Treino
color = 'tab:blue'
ax1.set_xlabel('Época')
ax1.set_ylabel('Perda de Treino (Loss)', color=color)
ax1.plot(loss_hist, color=color, label='Perda de Treino')
ax1.tick_params(axis='y', labelcolor=color)

# Cria o segundo eixo (mAP de Validação) que compartilha o eixo x
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('mAP de Validação', color=color)
ax2.plot(map_hist, color=color, label='mAP de Validação')
ax2.tick_params(axis='y', labelcolor=color)

# Título e legenda
plt.title('Performance do Modelo: Perda de Treino vs. mAP de Validação')
fig.tight_layout()
fig.legend(loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax1.transAxes)

# Salva o gráfico no seu Drive
plt.savefig(os.path.join(CHECKPOINT_DIR, "loss_vs_map_plot.png"))
plt.show()